# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
nltk.download(['punkt', 'wordnet','stopwords','averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     /home/giuliocesare/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/giuliocesare/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/giuliocesare/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/giuliocesare/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def remove_outliers(df):
    
    text_length = df['message'].apply(lambda x: len(x))
    outliers = df['message'][text_length[text_length < 25].index].values
    df = df[~df['message'].isin(outliers)]
    
    return df

In [3]:
# load data from database
engine = create_engine('sqlite:///./data/ModelingData.db')
df = pd.read_sql("SELECT * FROM ModelingData", engine)
df = remove_outliers(df)
columns = list(df.iloc[:,4:].columns)
X = df['message'].values
y = df.iloc[:,4:].values

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # get list of all urls using regex
    url_regex = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    repeated_symbols_regex = r"[\?\.\!]+(?=[\?\.\!])"
    
    text = re.sub(url_regex,"urlplaceholder",text)
    text = re.sub(repeated_symbols_regex,'', text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    #lemmatizer and stopwords
    clean_tokens = [WordNetLemmatizer().lemmatize(w.lower().strip())
                    for w in tokens if w not in stopwords.words('english')]
    
    #Stemmer
    clean_tokens = [PorterStemmer().stem(t) for t in clean_tokens]
    
    #Removing Symbols
    symbols_list = ['_','-','?','!','.','@','#','$','%','^','&','*','(',')','[',']','/']
    clean_tokens = [PorterStemmer().stem(t) for t in clean_tokens if t not in symbols_list]
    
    return clean_tokens

In [5]:
tokenize(X[0])

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pa', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1),n_jobs=-1)),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
pipeline.fit(X_train,y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [174]:
y_pred = pipeline.predict(X_test)

In [175]:
for i in range(y_pred.shape[1]):
    print(f"Feature Name: {columns[i]}")
    print(classification_report(y_test[:,i], y_pred[:,i]))

Feature Name: related
              precision    recall  f1-score   support

           0       0.63      0.44      0.52      1873
           1       0.83      0.92      0.87      5934
           2       0.67      0.07      0.12        58

   micro avg       0.80      0.80      0.80      7865
   macro avg       0.71      0.48      0.51      7865
weighted avg       0.78      0.80      0.78      7865

Feature Name: request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      6533
           1       0.78      0.41      0.54      1332

   micro avg       0.88      0.88      0.88      7865
   macro avg       0.83      0.69      0.73      7865
weighted avg       0.87      0.88      0.86      7865

Feature Name: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

   micro avg       1.00      1.00      1.00      7865
   macro a

/anaconda3/envs/data_scientist/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.94      0.98      0.96      7127
           1       0.72      0.37      0.49       738

   micro avg       0.93      0.93      0.93      7865
   macro avg       0.83      0.68      0.73      7865
weighted avg       0.92      0.93      0.92      7865

Feature Name: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7782
           1       0.00      0.00      0.00        83

   micro avg       0.99      0.99      0.99      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.98      0.99      0.98      7865

Feature Name: earthquake
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7163
           1       0.88      0.71      0.78       702

   micro avg       0.97      0.97      0.97      7865
   macro avg       0.93      0.85      0.88      7865
weighted avg       0.96      0

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {'clf__estimator__min_samples_split': [2, 3]}

In [16]:
param_distributions = {'clf__estimator__n_estimators': [10],
                       'clf__estimator__criterion':['gini','entropy'],
                       'clf__estimator__max_depth':list(range(1,10))+list(range(10,100,10))+list(range(100,1100,100)),
                       'clf__estimator__min_samples_split': list(range(2,20)),
                       'clf__estimator__min_samples_leaf': list(range(2,20))}

In [9]:
cv = GridSearchCV(pipeline, 
                  param_grid=parameters,
                  cv=3,
                  n_jobs=-1)

In [20]:
cv = RandomizedSearchCV(pipeline,
                        param_distributions=param_distributions,
                        n_iter=20,
                        cv=3,
                        n_jobs=-1)

In [21]:
cv.fit(X_train,y_train)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              CountVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.int64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                          

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
clf = cv.best_estimator_
y_pred = clf.predict(X_test)

In [23]:
for i in range(y_pred.shape[1]):
    print(f"Feature Name: {columns[i]}")
    print(classification_report(y_test[:,i], y_pred[:,i]))

Feature Name: related
              precision    recall  f1-score   support

           0       0.71      0.37      0.49      1232
           1       0.83      0.95      0.88      3975
           2       1.00      0.15      0.26        34

    accuracy                           0.81      5241
   macro avg       0.84      0.49      0.54      5241
weighted avg       0.80      0.81      0.79      5241

Feature Name: request
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      4378
           1       0.82      0.31      0.45       863

    accuracy                           0.88      5241
   macro avg       0.85      0.65      0.69      5241
weighted avg       0.87      0.88      0.85      5241

Feature Name: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5210
           1       0.00      0.00      0.00        31

    accuracy                           0.99      5241
   macro a

/home/giuliocesare/anaconda3/envs/disaster_response/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4953
           1       0.63      0.04      0.08       288

    accuracy                           0.95      5241
   macro avg       0.79      0.52      0.53      5241
weighted avg       0.93      0.95      0.92      5241

Feature Name: electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5128
           1       1.00      0.04      0.07       113

    accuracy                           0.98      5241
   macro avg       0.99      0.52      0.53      5241
weighted avg       0.98      0.98      0.97      5241

Feature Name: tools
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        23

    accuracy                           1.00      5241
   macro avg       0.50      0.50      0.50      5241
weighted avg       0.99     

In [25]:
from joblib import dump, load

In [26]:
dump(cv, 'cv_clf.joblib') 

['cv_clf.joblib']

In [ ]:
cv = load('filename.joblib') 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [6]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except:
                pass
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
def build_model(parameters):
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

In [ ]:
parameters = {'clf__estimator__n_estimators': [50],
              'clf__estimator__min_samples_split': [2, 3, 4]}

cv = build_model(parameters)

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.